In [ ]:
import sys
from pathlib import Path

# Aggiungere la directory src al path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from models import UNet
from losses import CombinedLoss
from training import get_dataloaders, run_training
from utils import (
    setup_experiment,
    resume_training,
    get_degraded_data_dir,
    get_raw_data_dir,
)

## Configurazione con Resume Flag

In [ ]:
# ⚙️ CONFIGURATION
config = {
    # 🔄 RESUME TRAINING - Imposta questo flag!
    "resume_from_checkpoint": True,   # ← Cambia a True per riprendere training
    "resume_experiment": "latest",     # "latest" o timestamp specifico come "20251229_224726"
    
    # Data paths
    "train_degraded_dir": str(get_degraded_data_dir() / "gaussian" / "DIV2K_train_HR"),
    "train_clean_dir": str(get_raw_data_dir() / "DIV2K_train_HR"),
    "val_degraded_dir": str(get_degraded_data_dir() / "gaussian" / "DIV2K_valid_HR"),
    "val_clean_dir": str(get_raw_data_dir() / "DIV2K_valid_HR"),
    
    # Training hyperparameters
    "batch_size": 16,
    "num_epochs": 50,  # ← Totale epoche (include quelle già fatte se resume)
    "learning_rate": 1e-4,
    "weight_decay": 1e-5,
    
    # Model
    "model_features": 64,
    "model_bilinear": True,
    
    # Optimizer
    "scheduler": "cosine",
    "warmup_epochs": 5,
    "min_lr": 1e-6,
    "patience": 5,
    
    # Device
    "device": "cuda" if torch.cuda.is_available() else "cpu",
}

print("📋 Configurazione:")
print(f"   Resume from checkpoint: {config['resume_from_checkpoint']}")
if config['resume_from_checkpoint']:
    print(f"   Resume experiment: {config['resume_experiment']}")
print(f"   Device: {config['device']}")
print(f"   Total epochs: {config['num_epochs']}")

## Setup Esperimento

In [ ]:
# Se non stai facendo resume, crea un nuovo esperimento
# (altrimenti lo riutilizzerai quello esistente)
if not config["resume_from_checkpoint"]:
    exp_dir, subdirs = setup_experiment(
        model_name="unet",
        degradation="gaussian",
        config=config,
        custom_name="v1"
    )
    checkpoints_dir = subdirs["checkpoints"]
    logs_dir = subdirs["logs"]
else:
    # Quando fai resume, userai le directory dell'esperimento esistente
    print("⏭️  Skipping new experiment setup (will use existing from resume)")

## Crea DataLoaders

In [ ]:
train_loader, val_loader = get_dataloaders(
    train_degraded_dir=config["train_degraded_dir"],
    train_clean_dir=config["train_clean_dir"],
    val_degraded_dir=config["val_degraded_dir"],
    val_clean_dir=config["val_clean_dir"],
    batch_size=config["batch_size"],
    patch_size=128,
    patches_per_image=10,
    num_workers=4,
)

print(f"✅ DataLoaders created: {len(train_loader)} train batches, {len(val_loader)} val batches")

## Inizializza Model, Optimizer, Scheduler

In [ ]:
# Crea modello, optimizer e scheduler come al solito
model = UNet(
    in_channels=3,
    out_channels=3,
    features=config["model_features"],
    bilinear=config["model_bilinear"],
).to(config["device"])

criterion = CombinedLoss(alpha=0.84, beta=0.16).to(config["device"])

optimizer = optim.AdamW(
    model.parameters(),
    lr=config["learning_rate"],
    weight_decay=config["weight_decay"]
)

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=config["num_epochs"] - config["warmup_epochs"],
    eta_min=config["min_lr"],
)

print(f"✅ Model created: {model.get_num_params():,} parameters")

## 🔄 Resume Logic - QUESTO È IL PUNTO CHIAVE!

In [ ]:
# Variabili per tracking
start_epoch = 0
initial_best_loss = float("inf")

if config["resume_from_checkpoint"]:
    # 🔄 RESUME: Carica tutto automaticamente
    checkpoint_info, start_epoch = resume_training(
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        experiment_path=config["resume_experiment"],
        model_name="unet",
        degradation="gaussian",
        device=config["device"],
    )
    
    # Estrai best loss precedente
    initial_best_loss = checkpoint_info["metrics"].get("val", {}).get("loss", float("inf"))
    
    # Usa le directory dell'esperimento caricato
    # (estrai il path dall'esperimento caricato)
    from utils import get_model_experiments_dir
    base_dir = get_model_experiments_dir("unet", "gaussian")
    
    if config["resume_experiment"] == "latest":
        experiment_dirs = sorted(
            [d for d in base_dir.iterdir() if d.is_dir()],
            key=lambda x: x.name,
            reverse=True,
        )
        exp_dir = experiment_dirs[0]
    else:
        exp_dir = base_dir / config["resume_experiment"]
    
    checkpoints_dir = exp_dir / "checkpoints"
    logs_dir = exp_dir / "logs"
    
    print(f"\n✅ Resuming from: {exp_dir}")
    print(f"   Start epoch: {start_epoch}")
    print(f"   Previous best loss: {initial_best_loss:.4f}")
    
else:
    # 🆕 NUOVO TRAINING
    print("\n🆕 Starting fresh training")
    print(f"   Experiment: {exp_dir}")

## TensorBoard Setup

In [ ]:
writer = SummaryWriter(log_dir=logs_dir)
print(f"📊 TensorBoard logs: {logs_dir}")

## Run Training

In [ ]:
# Run training con i parametri di resume
history, best_info = run_training(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=config["device"],
    num_epochs=config["num_epochs"],
    checkpoints_dir=checkpoints_dir,
    writer=writer,
    warmup_epochs=config["warmup_epochs"],
    learning_rate=config["learning_rate"],
    patience=config["patience"],
    save_every=5,
    val_every=1,
    # ⬇️ PARAMETRI CRITICI PER RESUME
    start_epoch=start_epoch,
    initial_best_loss=initial_best_loss,
)

print("\n✅ Training completato!")
print(f"   Best epoch: {best_info['best_epoch']}")
print(f"   Best loss: {best_info['best_val_loss']:.4f}")

## Riepilogo

### Per iniziare un nuovo training:
```python
config["resume_from_checkpoint"] = False
```

### Per riprendere l'ultimo training:
```python
config["resume_from_checkpoint"] = True
config["resume_experiment"] = "latest"
```

### Per riprendere un training specifico:
```python
config["resume_from_checkpoint"] = True
config["resume_experiment"] = "20251229_224726"  # timestamp specifico
```

**Tutto viene caricato automaticamente**: modello, optimizer, scheduler, epoch, best loss! 🎉